# Temperature forcasting with RNNS

Download data from bgc institut in jenna.

In [1]:
# !wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
# !curl -O https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
# !unzip jena_climate_2009_20016.csv  

In [2]:
import os 

data_dir = 'datasets/'
fname = os.path.join(data_dir,'jena_climate_2009_2016.csv')

with open(fname) as F:
    data= F.read()
# seperate lines from heder 
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
# Print header and length.
print('\n'.join(header))
print(f"The data contains {len(lines)} samples.")

"Date Time"
"p (mbar)"
"T (degC)"
"Tpot (K)"
"Tdew (degC)"
"rh (%)"
"VPmax (mbar)"
"VPact (mbar)"
"VPdef (mbar)"
"sh (g/kg)"
"H2OC (mmol/mol)"
"rho (g/m**3)"
"wv (m/s)"
"max. wv (m/s)"
"wd (deg)"
The data contains 420551 samples.


In [3]:
import numpy as np

float_data = np.zeros((len(lines), len(header)-1))
for i, line in enumerate(lines):
    values = [float(x)for x in line.split(',')[1:]]
    float_data[i, :] = values 

In [4]:
from matplotlib import pyplot as plt
temp = float_data[:, 1]
plt.plot(temp)
plt.title('Temperature')
plt.figure()
plt.plot(np.arange(0,1440)*10/60/24, temp[:1440])
plt.title('Temperature in the first 10 days')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

* lookback = 720 - 5 days
* steps = 6 -hourly sample rate(the data comes in 10 min intervals)
* delay 144 - predict next 24 hours

Since we will be lookingat the data throughasliding window we will use agenerator as not to duplicate the data(although numpy would probably take care of this).

In [5]:
# 0:200000 -training data
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std 

In [17]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i= min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback,
                                    max_index, size=batch_size)
        else:
            if i + batchsize >= max_index:
                i= min_index + lookback
            rows = np.arange(i, min(i + batchsize,max_index))
            i += len(rows)
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1] # Only temperature is the target.
        yield samples, targets

In [65]:
lookback = 1440
step = 6
delay= 144
batch_size = 128
train_gen = generator(float_data, lookback=lookback, delay=delay,
                      min_index=0, max_index=200000, shuffle=True,
                      step=step, batch_size=batch_size)
val_gen   = generator(float_data, lookback=lookback, delay=delay,
                      min_index=200001, max_index=300000, shuffle=True,
                      step=step, batch_size=batch_size)
test_gen = generator(float_data, lookback=lookback, delay=delay,
                      min_index=300001, max_index=None, shuffle=True,
                      step=step, batch_size=batch_size)
val_steps = (300000 - 200001 - lookback)
val_steps = (len(float_data) - 300001 - lookback)

In [69]:
# Common sense baseline:
#   Supose temperature is the same as before 24 h.

def evaluate_naive_method():
    batch_maes = []
    for step in range(3000):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(f"Naive MAE {np.mean(batch_maes)}")
    print(f"Navie MAE in C {np.mean(batch_maes) * std[1]}")
evaluate_naive_method()
    

Naive MAE 0.28849283149608074
Navie MAE in C 2.553882526041667


In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Flatten(input_shape=(lookback//step,float_data.shape[-1])))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(),loss='mae')
history = model.fit_generator(train_gen,
                             steps_per_epoch=500,
                             epochs=20,
                             validation_data=val_gen,
                             validation_steps=val_steps)

In [ ]:
def plot_history(history):
    def plot_history_record(history, record, *args, **kwargs):
        values = history.history[record]
        epochs = range(1, len(values) + 1)
        plt.plot(epochs, values, *args, **kwargs)

    plot_history_record(history, 'loss', 'bo', label='Training loss')
    plot_history_record(history, 'val_loss', 'b', label='Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.figure()
    plot_history_record(history, 'acc', 'bo', label='Training accuracy')
    plot_history_record(history, 'val_acc', 'b', label='Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
plot_history(history)

In [82]:
samples, targets = next(val_gen)
samples.shape

(128, 240, 14)

TypeError: cannot unpack non-iterable int object